# Grid Fluid

Modeling fluid motion by solving the incompressible and inviscid Navier-Stokes equation on a uniform grid

Credit to Bo Zhu, on whose Dartmouth COSC89 course notes the following code has been based.

Additionally, credit to Robert Bridson from the University of British Columbia, whose Fluid Simulation Course Notes were invaluable in understanding the material. His course notes can be found here:\
https://www.cs.ubc.ca/~rbridson/fluidsimulation/fluids_notes.pdf

In [1]:
import taichi as ti
import taichi.math as tm
import numpy as np

ti.init()

[Taichi] version 1.2.1, llvm 10.0.0, commit 12ab828a, osx, python 3.9.7
[I 11/16/22 13:14:16.482 759730] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=x64


In [2]:
### Global Variables ###

iters = 40
dt = 0.03
res = 300
# dx = 0.015
dx = 1.0 # /res

vel = ti.Vector.field(2, float, shape=(res, res))
vel_copy = ti.Vector.field(2, float, shape=(res,res))
vel_divs = ti.field(float, shape=(res, res))
pressure_hat = ti.field(float, shape=(res, res))
smoke_density = ti.field(float, shape=(res,res))
den_copy = ti.field(float, shape=(res,res))
vor = ti.field(float, shape=(res,res))
N = ti.Vector.field(2, float, shape=(res, res))

init_vel = tm.vec2(0.5,0)
new_src_vel = 3.0
new_src_rad = 0.05
src_pos = tm.vec2(0.5, 0.5)
src_vel = tm.vec2(3, 0)
src_rad = 0.1

colors = ti.Vector.field(3, float, shape=(res,res))

debug = False
source_follow_mouse = True

# tester variables
smoke_check = ti.field(float, shape=())
old_vel_check = ti.Vector.field(2, float, shape=())
vel_check = ti.Vector.field(2, float, shape=())
vel_check_surrounding = ti.Vector.field(2, float, shape=5)
new_pos_check = ti.Vector.field(2, float, shape=())
vor_check = ti.field(float, shape=())
f_N_check = ti.Vector.field(2, float, shape=())
pressure_check = ti.field(float, shape=())
div_check = ti.field(float, shape=())
lap_check = ti.field(float, shape=())
pressure_surrounding_check = ti.field(float, shape=5)
grad_check = ti.Vector.field(2, float, shape=())
iter_pressure_checks = ti.field(float, shape=iters)

lap_p = ti.field(float, shape=(res, res))
interpolate_test = ti.Vector.field(2, float, shape=2)
bound_check = ti.field(int, shape=4)
pre_check = ti.field(float, shape=())
post_check = ti.field(float, shape=())
compare_to_lap = ti.field(float, shape=())

clamp_check = ti.Vector.field(2, float, shape=())

In [18]:
### Helper Functions ###

# @param - coordinates (i,j) of the node
# @return - gradient of p at node (i,j)
@ti.func 
def gradient_pressure(i:ti.i32, j:ti.i32):
    pressures = tm.vec2(pressure_hat[i+1, j] - pressure_hat[i-1, j], pressure_hat[i, j+1] - pressure_hat[i, j-1])
    return pressures / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - divergence of the velocity at grid node (i, j)
@ti.func
def divergence_vel(i:ti.i32, j:ti.i32):
    return (vel[i+1, j][0] - vel[i-1, j][0] + vel[i, j+1][1] - vel[i, j-1][1]) / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - the curl of the velocity at grid node (i, j)
@ti.func
def curl_vel(i:ti.i32, j:ti.i32):
    return (vel[i+1, j][1] - vel[i-1, j][1] - vel[i, j+1][0] + vel[i, j-1][0]) / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - the laplacian of the pressure at grid node (i, j)
@ti.func
def laplacian_pressure(i:ti.i32, j:ti.i32):
    return (pressure_hat[i+1, j] + pressure_hat[i-1, j] + pressure_hat[i, j+1] + pressure_hat[i, j-1] - 4 * pressure_hat[i, j]) / (dx * dx)

# returns whether or not (row, col) is on a boundary
@ti.func
def isBoundary(row:ti.i32, col:ti.i32) -> bool:
    return (row == 0 or col == 0 or row == res-1 or col == res-1)

In [55]:
### Simulator Functions ###

# ensures that x_p is within the field
# Note: upper bound is res, not 1
@ti.func
def clamp_pos(pos):
    epsilon = dx * 1e-3
    clamped_pos = tm.vec2(pos[0], pos[1])
    if clamped_pos[0] < 0:
        clamped_pos[0] = epsilon
    if clamped_pos[1] < 0:
        clamped_pos[1] = epsilon
    if clamped_pos[0] >= res:
        clamped_pos[0] = res-epsilon
    if clamped_pos[1] >= res:
        clamped_pos[1] = res-epsilon
    return clamped_pos

@ti.func
def needs_clamping(x1, x2, y1, y2):
    return x1 < 0 or x2 < 0 or y1 < 0 or y2 < 0 or x1 >= res or x2 >= res or y1 >= res or y2 >= res

# @param - x_p: vec2, where x_p = (x - u^(n+1/2)(x) * dt) (the "starting point")
# @return - the interpolated value for u*(x) at x_p
@ti.func
def interpolate(x_p, vector_field):
    x_p = clamp_pos(x_p)
    x = x_p[0]
    y = x_p[1]
    x1 = (int)(ti.floor(x))
    x2 = x1+1
    y1 = (int)(ti.floor(y))
    y2 = y1+1
    # clamp pos
    if needs_clamping(x1, x2, y1, y2):
        if x1 < 0:
            x1 = 0
            x2 = 1
        if y1 < 0:
            y1 = 0
            y2 = 1
        if x2 >= res:
            x1 = res-2
            x2 = res-1
        if y2 >= res:
            y1 = res-2
            y2 = res-1
    # interpolate in x direction
    f_xy1 = (x2 - x)/(x2-x1) * vector_field[x1, y1] + (x - x1)/(x2 - x1) * vector_field[x2, y1]
    f_xy2 = (x2 - x)/(x2-x1) * vector_field[x1, y2] + (x - x1)/(x2 - x1) * vector_field[x2, y2]
    # interpolate in y direction
    f_xy = (y2 - y)/(y2 - y1) * f_xy1 + (y - y1)/(y2 - y1) * f_xy2
    
    bound_check[0] = x1
    bound_check[1] = x2
    bound_check[2] = y1
    bound_check[3] = y2
    
    return f_xy

# Fills vel_copy and den_copy
@ti.func
def fill_copies():
    for i in ti.grouped(vel):
        vel_copy[i] = vel[i]
        den_copy[i] = smoke_density[i]

# Advection using a semi-Lagrangian method
@ti.kernel
def advect():
    fill_copies()
    for row in range(res):
        for col in range(res):
            pos = tm.vec2(row, col)
            new_pos = pos - vel[row, col] * 0.5 * dt
            new_vel = interpolate(new_pos, vel_copy)
            new_pos = pos - new_vel * dt
            new_vel = interpolate(new_pos, vel_copy)
            vel[row, col] = new_vel
            
            new_den = interpolate(new_pos, den_copy)
            smoke_density[row, col] = new_den

@ti.func
def calc_divs():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vel_divs[row, col] = divergence_vel(row, col)
    
@ti.func 
def fill_lap_p():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                lap_p[row, col] = laplacian_pressure(row, col)
    
@ti.kernel
def project():
    # Calc velocity for RHS of Poissan (div u*)
    # step 1: calculate divergence on each node
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vel_divs[row, col] = divergence_vel(row, col)
                
    # step 2: solve the Poisson's equation -lap p= div u using the Gauss-Seidel iterations
    ti.loop_config(serialize=True)
    for i in range(iters):
        for row in range(res):
            for col in range(res):
                if not isBoundary(row, col):
                    coef_off_dia = 1.0 / (dx * dx)
                    coef_dia = 4.0 / (dx * dx)
                    off_diagonal = pressure_hat[row-1, col] + pressure_hat[row+1, col] + pressure_hat[row, col+1] + pressure_hat[row, col-1]
                    pressure_hat[row, col] = (-vel_divs[row, col] + off_diagonal * coef_off_dia) / coef_dia
        iter_pressure_checks[i] = pressure_hat[250, 200]
    
    fill_lap_p()
    pre_check[None] = vel_divs[250, 200]
    
    # step 3: correct velocity with the pressure gradient
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                grad_p = gradient_pressure(row, col)
                vel[row, col] -= grad_p
    
    calc_divs()
    post_check[None] = vel_divs[250, 200]
    compare_to_lap[None] = lap_p[250, 200]

@ti.func
def cross(v, w):
    return tm.vec2(v[1] * w, -v[0] * w)
                      
@ti.kernel
def vorticity_confinement():
    vor.fill(0.)
    # step 1: update vorticity
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vor[row, col] = curl_vel(row, col)

    # step 2: update N = (grad(|vor|)) / |grad(|vor|)|
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                N[row, col][0] = (abs(vor[row+1, col]) - abs(vor[row-1, col]))/ (2 * dx)
                N[row, col][1] = (abs(vor[row, col+1]) - abs(vor[row, col-1]))/ (2 * dx)
                N[row, col] /= (N[row, col].norm() + 1e-20)

    # step 3: calculate confinement force and use it to update velocity
    vor_conf_coef = 0.0 # 4.0
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                f = vor_conf_coef * dx * cross(N[row, col], vor[row, col])
                vel[row, col] += f * dt
    
# create a "source" for the smoke
@ti.kernel
def source(mouse_data: ti.types.ndarray()):
    center = tm.vec2(mouse_data[2]/res, mouse_data[3]/res)
    direction = tm.vec2(mouse_data[1], mouse_data[2])
    for row in range(res):
        for col in range(res):
            if (tm.vec2(row/res, col/res) - center).norm() < src_rad:
#                 vel[row, col] = src_vel
                vel[row, col] = direction
                smoke_density[row, col] = 1

# 0 = up, 1 = left, 2 = down, 3 = right
@ti.kernel
def directed_source(x:ti.f32, y:ti.f32, direction:ti.i32):
    new_source_pos = tm.vec2(x, y)
    for row in range(res):
        for col in range(res):
            if (tm.vec2(row/res, col/res) - new_source_pos).norm() < src_rad:
                if direction == 0: # up
                    vel[row, col] = tm.vec2(0, new_src_vel)
                elif direction == 1: # left
                    vel[row, col] = tm.vec2(-new_src_vel, 0)
                elif direction == 2: # down
                    vel[row, col] = tm.vec2(0, -new_src_vel)
                else: # right
                    vel[row, col] = tm.vec2(new_src_vel, 0)
                smoke_density[row, col] = 1

@ti.kernel
def update_colors():
    for i in ti.grouped(colors):
        curr_den = smoke_density[i]
        colors[i] = tm.vec3(curr_den, curr_den, curr_den)

In [56]:
def substep(mouse_data):
    # renew source for each substep if user asks for it
    if constant_source:
        source(mouse_data)
        print("Adding a new source at (%d, %d)" % (mouse_data[2]/res, mouse_data[3]/ res))
    advect()
    vorticity_confinement()
    project()
    update_colors()

In [57]:
### Setup ###
@ti.kernel
def init_grid():
    for i in ti.grouped(vel):
        vel[i] = init_vel
        N[i] = tm.vec2(0,0)
    smoke_density.fill(0.)
    vel_divs.fill(0.)
    pressure_hat.fill(0.)
    vor.fill(0.)
    interpolate_test[0] = interpolate(tm.vec2(250, 250), vel)
    interpolate_test[1] = interpolate(tm.vec2(250, 251), vel)

In [58]:
@ti.kernel
def valid_pos(x:ti.f32, y:ti.f32) -> ti.i32:
    valToReturn = 0 # start at false
    if x < 1 and x >= 0 and y < 1 and y >= 0:
        valToReturn = 1
    return valToReturn
    
### Function to get print checks ###
@ti.kernel
def get_checks(x:ti.f32, y:ti.f32):
    row = (int)(x * res)
    col = (int)(y * res)
#     row = (int)(pos[0])
#     col = (int)(pos[1])
    smoke_check[None] = smoke_density[row, col]
    vel_check[None] = vel[row, col]
    vor_check[None] = vor[row, col]
    pressure_check[None] = pressure_hat[row, col]
    f_N_check[None] = N[row, col]
#     vel_check_surrounding[0] = vel[row+1, col]
#     vel_check_surrounding[1] = vel[row-1, col]
#     vel_check_surrounding[2] = vel[row, col+1]
#     vel_check_surrounding[3] = vel[row, col-1]
#     vel_check_surrounding[4] = vel[row, col]
    vel_check_surrounding[0] = vel[251, 200]
    vel_check_surrounding[1] = vel[249, 200]
    vel_check_surrounding[2] = vel[250, 201]
    vel_check_surrounding[3] = vel[250, 199]
    vel_check_surrounding[4] = vel[250, 200]
    div_check[None] = vel_divs[row, col]
    lap_check[None] = lap_p[row, col]
#     pressure_surrounding_check[0] = pressure_hat[row+1, col]
#     pressure_surrounding_check[1] = pressure_hat[row-1, col]
#     pressure_surrounding_check[2] = pressure_hat[row, col+1]
#     pressure_surrounding_check[3] = pressure_hat[row, col-1]
#     pressure_surrounding_check[4] = pressure_hat[row, col]
    pressure_surrounding_check[0] = pressure_hat[251, 200]
    pressure_surrounding_check[1] = pressure_hat[249, 200]
    pressure_surrounding_check[2] = pressure_hat[250, 201]
    pressure_surrounding_check[3] = pressure_hat[250, 199]
    pressure_surrounding_check[4] = pressure_hat[250, 200]
    grad_check[None] = gradient_pressure(row, col)

def print_checks(x:ti.f32, y:ti.f32):
    row = (int)(x * res)
    col = (int)(y * res)
    print("(row, col) = (%d, %d)" % (row, col))
    print("smoke check", smoke_check[None])
    print("vel check", vel_check[None])
    print("vor check", vor_check[None])
    print("N check", f_N_check[None])
    print("pressure check", pressure_check[None])
    print("pressure surrounding", pressure_surrounding_check)
    print("div check", div_check[None])
    print("lap check", lap_check[None])
    print("surrounding velocities", vel_check_surrounding)
    print("grad check", grad_check)
    print("pre", pre_check[None])
    print("post", post_check[None])
    print("compare to lap", compare_to_lap[None])
    print("iter pressure checks", iter_pressure_checks)


In [59]:
class MouseData:
    def __init__(self):
        self.prev_mouse = None
    
    def __call__(self, gui):
        # [0:2]: normalized delta direction
        # [2:4]: current mouse xy
        mouse_data = np.zeros(8, dtype=np.float32)
        if gui.is_pressed(ti.GUI.LMB):
            mouse_xy = np.array(gui.get_cursor_pos(), dtype=np.float32) * res
            if self.prev_mouse is None:
                self.prev_mouse = mouse_xy
            else:
                mouse_dir = mouse_xy - self.prev_mouse
                mouse_dir = mouse_dir / (np.linalg.norm(mouse_dir) + 1e-5)
                mouse_data[0], mouse_data[1] = mouse_dir[0], mouse_dir[1]
                mouse_data[2], mouse_data[3] = mouse_xy[0], mouse_xy[1]
                self.prev_mouse = mouse_xy
        else:
            self.prev_mouse = None
        return mouse_data

In [62]:
def main():
    gui = ti.GUI('Grid Fluid', (res, res))
    md_gen = MouseData()
    
    visualize_smoke = True  #visualize density (default)
    visualize_vel = False  #visualize velocity
    visualize_div = False # visualize divergence
    visualize_curl = False # visualize curl
    
    global constant_source
    constant_source = False
        
    init_grid()
#     print("interpolate test", interpolate_test)
#     print("vel in middle", vel[250, 250])
#     print("bound check", bound_check)

#     print(clamp_check[None])
    
#     if not constant_source:
#         source(0.5, 0.5)
    
    if debug:
        mouse_data = md_gen(gui)
        substep(mouse_data)
        substep(mouse_data)
        substep(mouse_data)
        get_checks(0.5, 0.5)
        print_checks(0.5, 0.5)
        print("NEW STEP")
        substep(mouse_data)
        get_checks(0.5, 0.5)
        print_checks(0.5, 0.5)

    while gui.running:
        
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.SPACE:
                constant_source = not constant_source
                print("constant_source=", constant_source)
            elif e.key == 'w': # add a new directed source
                directed_source(e.pos[0], e.pos[1], 0)
            elif e.key == 'a':
                directed_source(e.pos[0], e.pos[1], 1)
            elif e.key == 's':
                directed_source(e.pos[0], e.pos[1], 2)
            elif e.key == 'd':
                directed_source(e.pos[0], e.pos[1], 3)
            elif e.key == ti.GUI.LMB:
                print("row", e.pos[0])
                if valid_pos(e.pos[0], e.pos[1]):
                    get_checks(e.pos[0], e.pos[1])
                    print_checks(e.pos[0], e.pos[1])
                    ###
#                     constant_source = False
#                     source(e.pos[0], e.pos[1])
            elif e.key == 'v': # show velocities
                visualize_vel = True
                visualize_smoke = False
                visualize_div = False
                visualize_curl = False
            elif e.key == 'b':
                visualize_smoke = True
                visualize_vel = False
                visualize_div = False
                visualize_curl = False
            elif e.key == 'g':
                visualize_smoke = False
                visualize_vel = False
                visualize_div = True
                visualize_curl = False
            elif e.key == 'h':
                visualize_smoke = False
                visualize_vel = False
                visualize_div = False
                visualize_curl = True
            elif e.key == 'r':
                init_grid()
             
        # Move stuff
        if not debug:
            mouse_data = md_gen(gui)
#             print("MOUSE DATA", mouse_data)
            substep(mouse_data)

        
#         print("div u", max((abs(vel_divs.to_numpy().reshape(-1)))))
#         print("lap p", max((abs(lap_p.to_numpy().reshape(-1)))))
#         print("diff", max((abs(vel_divs.to_numpy().reshape(-1))) - max((abs(lap_p.to_numpy().reshape(-1))))))

            
        # Draw grid
        if visualize_smoke:
            gui.set_image(colors)
        elif visualize_vel:
            gui.set_image(vel.to_numpy() * 0.01 + 0.5)
        elif visualize_div:
            gui.set_image(vel_divs.to_numpy() * 0.1 + 0.3)
        elif visualize_curl:
            gui.set_image(vor.to_numpy() * 0.03 + 0.5)
        gui.show()
        
    
if __name__ == '__main__':
    main()

row 0.3633333444595337
(row, col) = (109, 142)
smoke check 0.0
vel check [0.5 0. ]
vor check 0.0
N check [0. 0.]
pressure check 0.0
pressure surrounding [0. 0. 0. 0. 0.]
div check 0.0
lap check 0.0
surrounding velocities [[0.5 0. ]
 [0.5 0. ]
 [0.5 0. ]
 [0.5 0. ]
 [0.5 0. ]]
grad check [0. 0.]
pre 0.0
post 0.0
compare to lap 0.0
iter pressure checks [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
constant_source= True
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at

Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a n

Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a n

Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a n

Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a n

Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a n

Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a n

Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a n

Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a n

Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a new source at (0, 0)
Adding a n

## Todo
